# 1- Load the required modules

In [29]:
import numpy as np                     # Llibreria matemÃ tica
import pandas as pd
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mÃ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb         # Per fer servir el knn
%matplotlib inline

# 2- Load the dataset

In [31]:
twigen = pd.read_csv("datapreprocessed.csv", encoding='latin1')
twigen.head()

,gender,created,description_length,fav_number,link_color,name_length,retweet_count,sidebar_color,text_length,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
0,male,early_morning,21,0,cyan,7,0,white,109,False,110964,midday,Unknown,Chennai
1,male,midday,62,68,gray,11,0,white,139,False,7471,midday,Unknown,Eastern Time (US & Canada)
2,male,morning,35,7696,gray,14,1,white,80,False,5617,midday,India,Belgrade
3,male,evening,146,202,gray,11,0,white,138,False,1693,midday,United States,Pacific Time (US & Canada)
4,female,midday,160,37318,lightblue,12,0,black,95,False,31462,midday,Unknown,Unknown


# 3- Pre-processing

## Expand categoric variables to binary values

In [32]:
def expand_categoric_column_with_binary_values(column_name):
    global twigen
    print(column_name)
    print(twigen[column_name].unique())
    twigen = pd.concat([
            twigen.drop(column_name, axis=1),
            pd.get_dummies(twigen[column_name], prefix=column_name, sparse=True)
        ], axis=1)

expand_categoric_column_with_binary_values('created')
expand_categoric_column_with_binary_values('tweet_created')
expand_categoric_column_with_binary_values('link_color')
expand_categoric_column_with_binary_values('sidebar_color')
expand_categoric_column_with_binary_values('tweet_location')
expand_categoric_column_with_binary_values('user_timezone')

twigen.info()
print(twigen.columns)
twigen.head()

created
['early_morning' 'midday' 'morning' 'evening' 'afternoon']
tweet_created
['midday']
link_color
['cyan' 'gray' 'lightblue' 'lightred' 'blue' 'black' 'violet' 'red'
 'orange' 'brown' 'green' 'lightgreen' 'white' 'yellow']
sidebar_color
['white' 'black' 'lightblue' 'orange' 'lightgreen' 'gray' 'lightred' 'cyan'
 'red' 'brown' 'blue' 'yellow' 'violet' 'green']
tweet_location
['Unknown' 'India' 'United States' 'United Kingdom' 'Canada' 'Haiti'
 'Saudi Arabia' 'Venezuela' 'Italy' 'Malaysia' 'Ireland' 'Poland' 'Spain'
 'Belgium' 'Nigeria' 'Portugal' 'South Africa' 'New Zealand' 'France'
 'Lebanon' 'Dominican Republic' 'Germany' 'Australia' 'Czechia' 'Croatia'
 'Indonesia' 'Swaziland' 'Paraguay' 'Mexico' 'Ecuador' 'South Korea'
 'Turkey' 'Syria' 'Greenland' 'Botswana' 'Brazil' 'Tunisia' 'Colombia'
 'Greece' 'Trinidad and Tobago' 'Bangladesh' 'Somalia' 'Netherlands'
 'Sweden' 'Hong Kong' 'Argentina' 'Japan' 'Egypt' 'Israel' 'Bolivia'
 'Bosnia & Herzegovina' 'Sri Lanka' 'North Africa' 'U

,gender,description_length,fav_number,name_length,retweet_count,text_length,tweet_coord,tweet_count,created_afternoon,created_early_morning,...,user_timezone_Urumqi,user_timezone_Vienna,user_timezone_Vilnius,user_timezone_Volgograd,user_timezone_Warsaw,user_timezone_Wellington,user_timezone_West Central Africa,user_timezone_Yakutsk,user_timezone_Yerevan,user_timezone_Zagreb
0,male,21,0,7,0,109,False,110964,0,1,...,0,0,0,0,0,0,0,0,0,0
1,male,62,68,11,0,139,False,7471,0,0,...,0,0,0,0,0,0,0,0,0,0
2,male,35,7696,14,1,80,False,5617,0,0,...,0,0,0,0,0,0,0,0,0,0
3,male,146,202,11,0,138,False,1693,0,0,...,0,0,0,0,0,0,0,0,0,0
4,female,160,37318,12,0,95,False,31462,0,0,...,0,0,0,0,0,0,0,0,0,0


## Convert boolean columns to binary values

In [33]:
def boolean_to_binary(column_name):
    global twigen
    twigen[column_name] = twigen[column_name].apply(lambda boolean: int(boolean))

boolean_to_binary('tweet_coord')

twigen.tweet_coord.sum()

157

## Separate data from label

In [35]:
def parse_label(gender):
    if gender == 'female': return 0
    if gender == 'male': return 1
    if gender == 'brand': return 2
    assert False # ensure gender is valid

# All values in KNN must be numeric, label included
labels = twigen.gender.apply(parse_label)
data = twigen.drop('gender', axis=1)

print(data.shape)
data.head()

(18836, 331)


,description_length,fav_number,name_length,retweet_count,text_length,tweet_coord,tweet_count,created_afternoon,created_early_morning,created_evening,...,user_timezone_Urumqi,user_timezone_Vienna,user_timezone_Vilnius,user_timezone_Volgograd,user_timezone_Warsaw,user_timezone_Wellington,user_timezone_West Central Africa,user_timezone_Yakutsk,user_timezone_Yerevan,user_timezone_Zagreb
0,21,0,7,0,109,0,110964,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,62,68,11,0,139,0,7471,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35,7696,14,1,80,0,5617,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,146,202,11,0,138,0,1693,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,160,37318,12,0,95,0,31462,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 3- Data cross-validation

In [36]:
# Cross-validation: split data into training and test sets (test 30% of data)
data_train, data_test, labels_train, labels_test = cv.train_test_split(data, labels, test_size=.3, random_state=1)

## Training

In [37]:
data_train.info()
twigen.iloc[data_train.head().index]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13185 entries, 17267 to 235
Columns: 331 entries, description_length to user_timezone_Zagreb
dtypes: int64(7), uint8(324)
memory usage: 4.9 MB


,gender,description_length,fav_number,name_length,retweet_count,text_length,tweet_coord,tweet_count,created_afternoon,created_early_morning,...,user_timezone_Urumqi,user_timezone_Vienna,user_timezone_Vilnius,user_timezone_Volgograd,user_timezone_Warsaw,user_timezone_Wellington,user_timezone_West Central Africa,user_timezone_Yakutsk,user_timezone_Yerevan,user_timezone_Zagreb
17267,brand,81,0,14,0,134,0,7014,0,0,...,0,0,0,0,0,0,0,0,0,0
14303,male,159,249,14,0,137,0,359,0,0,...,0,0,0,0,0,0,0,0,0,0
11018,female,3,14271,9,0,110,0,87773,0,0,...,0,0,0,0,0,0,0,0,0,0
9068,male,33,19357,14,1,42,0,17575,0,0,...,0,0,0,0,0,0,0,0,0,0
8696,female,97,40,7,0,57,0,18238,0,0,...,0,0,0,0,0,0,0,0,0,0


## Test

In [38]:
data_test.info()
twigen.iloc[data_test.head().index]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5651 entries, 11871 to 18258
Columns: 331 entries, description_length to user_timezone_Zagreb
dtypes: int64(7), uint8(324)
memory usage: 2.1 MB


,gender,description_length,fav_number,name_length,retweet_count,text_length,tweet_coord,tweet_count,created_afternoon,created_early_morning,...,user_timezone_Urumqi,user_timezone_Vienna,user_timezone_Vilnius,user_timezone_Volgograd,user_timezone_Warsaw,user_timezone_Wellington,user_timezone_West Central Africa,user_timezone_Yakutsk,user_timezone_Yerevan,user_timezone_Zagreb
11871,female,3,69,12,0,98,0,12264,0,0,...,0,0,0,0,0,0,0,0,0,0
7136,brand,3,0,9,0,54,0,79914,0,0,...,0,0,0,0,0,0,0,0,0,0
13818,male,46,6021,9,1,91,0,49095,0,1,...,0,0,0,0,0,0,0,0,0,0
5231,brand,79,0,11,0,125,0,47600,0,1,...,0,0,0,0,0,0,0,0,0,0
1005,brand,145,90,11,0,99,0,3938,1,0,...,0,0,0,0,0,0,0,0,0,0


## Fitting

In [39]:
# Create a kNN classifier object
knc = nb.KNeighborsClassifier()

# Train the classifier
knc.fit(data_train, labels_train)

# Obtain accuracy score of learned classifier on test data
print('Accuracy')
print(knc.score(data_test, labels_test))

Accuracy
0.468766589984
